In [96]:
import pandas as pd
import datetime
from pprint import pprint


In [97]:
data = pd.read_excel('Выгрузка_пациентов.xlsx')
data['Дата приема'] = pd.to_datetime(data['Дата приема']).dt.date
data = data.sort_values(by=['Номер карты', 'Дата приема']).reset_index(drop=True)

In [98]:
dct = {}
for card in set(data['Номер карты']):
    dct.setdefault(card, 0)

for key in dct.keys():
    temp_data = data[data['Номер карты'] == key][['Дата приема', 'СПУ']].reset_index(drop=True)
    dct[key] = {temp_data.iat[0,0] : temp_data.iat[0,1]}
    for index in temp_data.index[1:]:
        if temp_data.iat[index, 1] not in dct[key].values():
            dct[key][temp_data.iat[index, 0]] =  temp_data.iat[index, 1]
        
 

In [99]:
final_data = data[data['Дата приема'] >= datetime.date(2018, 10, 2)].reset_index(drop=True)

In [100]:
pacients = pd.read_excel('Пациенты_для_проверки.xlsx')

In [101]:
final_data = final_data.merge(pacients[['Номер карты', 'Новая СПУ', 'Статус']], how='left')

In [102]:
final_data['СПУ'] = final_data['СПУ'].apply(lambda x: 'Скидка 5%' if x == 'Скидка 2ГИС' else 'Скидка 25%' if x == 'XXI век Друзья' else x)
final_data['Новая СПУ'] = final_data['Новая СПУ'].apply(lambda x: f'Скидка {str(int(x * 100))}%')

In [104]:
final_data['Проверка_СПУ'] = final_data['СПУ'] == final_data['Новая СПУ']

In [107]:
final_data.to_excel('Пациенты_проверка_СПУ.xlsx', index=False)